### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** if you having trouble displaying the maps try running `jupyter nbextension enable --py gmaps` in your environment and retry.

* Create a heat map that displays the humidity for every city from the part I of the homework.

  ![heatmap](Images/heatmap.png)

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

  ![hotel map](Images/hotel_map.png)

As final considerations:

* Create a new GitHub repository for this project called `API-Challenge` (note the kebab-case). **Do not add to an existing repo**
* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* For Part I, you must include a written description of three observable trends based on the data.
* You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
* For max intensity in the heat map, try setting it to the highest humidity found in the data set.
o
## Hints and Considerations

* The city data you generate is based on random coordinates as well as different query times; as such, your outputs will not be an exact match to the provided starter notebook.

* You may want to start this assignment by refreshing yourself on the [geographic coordinate system](http://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm).

* Next, spend the requisite time necessary to study the OpenWeatherMap API. Based on your initial study, you should be able to answer  basic questions about the API: Where do you request the API key? Which Weather API in particular will you need? What URL endpoints does it expect? What JSON structure does it respond with? Before you write a line of code, you should be aiming to have a crystal clear understanding of your intended outcome.

* A starter code for Citipy has been provided. However, if you're craving an extra challenge, push yourself to learn how it works: [citipy Python library](https://pypi.python.org/pypi/citipy). Before you try to incorporate the library into your analysis, start by creating simple test cases outside your main script to confirm that you are using it correctly. Too often, when introduced to a new library, students get bogged down by the most minor of errors -- spending hours investigating their entire code -- when, in fact, a simple and focused test would have shown their basic utilization of the library was wrong from the start. Don't let this be you!

* Part of our expectation in this challenge is that you will use critical thinking skills to understand how and why we're recommending the tools we are. What is Citipy for? Why would you use it in conjunction with the OpenWeatherMap API? How would you do so?

* In building your script, pay attention to the cities you are using in your query pool. Are you getting coverage of the full gamut of latitudes and longitudes? Or are you simply choosing 500 cities concentrated in one region of the world? Even if you were a geographic genius, simply rattling 500 cities based on your human selection would create a biased dataset. Be thinking of how you should counter this. (Hint: Consider the full range of latitudes).

* Once you have computed the linear regression for one chart, the process will be similar for all others. As a bonus, try to create a function that will create these charts based on different parameters.

* Remember that each coordinate will trigger a separate call to the Google API. If you're creating your own criteria to plan your vacation, try to reduce the results in your DataFrame to 10 or fewer cities.

* Lastly, remember -- this is a challenging activity. Push yourself! If you complete this task, then you can safely say that you've gained a strong mastery of the core foundations of data analytics and it will only go better from here. Good luck!

### Copyright

Trilogy Education Services © 2019. All Rights Reserved.



# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

import csv
import json
import urllib

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("clean_city_data.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mint hill,35.18,-80.65,84.99,70,20,3.31,US,1596730391
1,broken hill,-31.95,141.43,42.80,81,90,17.22,AU,1596730391
2,la maddalena,41.21,9.41,86.00,35,0,12.75,IT,1596730391
3,pangody,65.85,74.49,65.64,57,68,11.30,RU,1596730391
4,bluff,-46.60,168.33,49.78,88,98,5.46,NZ,1596730391
...,...,...,...,...,...,...,...,...,...
542,natal,-5.79,-35.21,82.40,65,75,10.29,BR,1596730423
543,cambridge,43.36,-80.31,72.00,56,1,2.77,CA,1596730278
544,makakilo city,21.35,-158.09,78.80,83,20,4.70,US,1596730423
545,sao bento do sul,-26.25,-49.38,64.99,54,0,3.00,BR,1596730423


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
lwr_bnd= float(input('what is your preference for the lowest temperature you desire?'))

what is your preference for the lowest temperature you desire?70


In [27]:
uppr_bnd = float(input("What is the hottest temperature you prefer?"))

What is the hottest temperature you prefer?79


In [28]:
# we have to make a DataFrame fitting all three conditions that are aksed for so first off:
# 1) we want a city with a temp <79 & > 70
# 2) with a wind chill < 10 mph
# 3) with zero Cloudiness so 
# I HAVE COMPILED CONDITIONS as weather_pref

weather_pref = city_data_df[(city_data_df['Max Temp']< uppr_bnd) & (city_data_df['Max Temp']> lwr_bnd) & 
                            (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] == 0)]
weather_pref

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,novikovo,51.15,37.89,76.66,60,0,7.34,RU,1596730392
46,micheweni,-4.97,39.83,75.87,75,0,9.69,TZ,1596730394
62,shakawe,-18.37,21.85,77.09,16,0,5.68,BW,1596730395
116,komsomolets,53.75,62.07,71.56,57,0,9.75,KZ,1596730251
155,castro,-24.79,-50.01,72.43,36,0,2.91,BR,1596730400
208,san vicente,-26.62,-54.13,78.46,32,0,5.46,AR,1596730403
223,cidreira,-30.18,-50.21,71.62,51,0,3.33,BR,1596730404
317,gimli,50.63,-96.99,75.99,70,0,4.00,CA,1596730349
323,kastamonu,41.50,33.67,72.37,46,0,4.90,TR,1596730350
362,maxixe,-23.86,35.35,71.60,68,0,9.17,MZ,1596730413


In [29]:
#configuring gmaps
gmaps.configure(api_key=g_key)

In [31]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = weather_pref[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=((39.837277,-74.088707)), zoom_level=1.5)



# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights= weather_pref["Max Temp"])
# Add the heatmap layer
fig.add_layer(heat_layer)

fig
# save fig
plt.savefig(('fig_pref_heat_map.png'))

<Figure size 432x288 with 0 Axes>

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = weather_pref

# Search for a hotel using google parameterization, radius from destnation city with specific weather criteria
params = {"radius": 5000,
          "type": "lodging",
          "key": g_key}


# We have to iterate through all hotels found within hotel_df, so use indes  
for index, row in hotel_df.iterrows():
    # so we can get the location based of the hotel_df (latitude and longitude)
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # i looked up specifics on how to focus on hotels, which like phones or highway signs it oten says
    #  "lodging" AND this helps us focus in on location via coordinates...using the methods we used in class acivities
    # we search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Then the search begins for 
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [ ]:
hotel_df
plt.savefig('hotels.pdf')

In [ ]:

hotel_df.shape

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# marker_layer to include locations of hotels visually
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
plt.savefig('fig_hotel_df.pdf')
fig




This Was Cool!!!!!